In [1]:
import os
import yaml
import mlflow
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path

c:\Users\angel\.conda\envs\vehicle-env\lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [2]:
import os
import yaml
import mlflow
from pathlib import Path
from ultralytics import YOLO


current_path = Path.cwd()
if current_path.name == "notebooks":
    project_root = current_path.parent
else:
    project_root = current_path

print(f"📂 Raíz del proyecto detectada: {project_root}")


parts_yaml_path = project_root / "data" / "processed" / "parts_dataset" / "dataset.yaml"
damages_yaml_path = project_root / "data" / "processed" / "damages_dataset" / "dataset.yaml"



📂 Raíz del proyecto detectada: c:\Users\angel\Desktop\Proyecto\vehicle-damage-assesment


In [3]:
import torch
import ultralytics
print(f"🖥️ ¿CUDA disponible?: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🚀 Tarjeta detectada: {torch.cuda.get_device_name(0)}")
else:
    print("🐢 ESTÁS USANDO CPU (Por eso va lento)")

🖥️ ¿CUDA disponible?: True
🚀 Tarjeta detectada: NVIDIA GeForce GTX 1660 Ti


In [4]:
EPOCHS = 20       
IMG_SIZE = 640     
BATCH_SIZE = 8     
PROJECT_NAME = "Lab_Vehicle_Parts"

experiments = [
    {
        "name": "Exp1_Baseline",
        "params": {
            "optimizer": "auto", # YOLO elige el mejor (suele ser AdamW)
            "lr0": 0.01,       # Learning Rate: La velocidad a la que aprende
            "mosaic": 1.0,     # Augmentation: 100% probabilidad de hacer collage
        }
    },
    {
        "name": "Exp2_SGD_Slow",
        "params": {
            "optimizer": "SGD", # Stochastic Gradient Descent (Clásico, lento pero preciso)
            "lr0": 0.005,      # 0.005 es la mitad de 0.01. Aprende más despacio.
            "mosaic": 1.0,
        }
    },
    {
        "name": "Exp3_Heavy_Augment",
        "params": {
            "optimizer": "auto",
            "lr0": 0.01,
            "mosaic": 0.0,     # APAGAMOS el collage (Mosaic)
            "fliplr": 0.8,     # ENCENDEMOS volteo horizontal (espejo) al 80%
        }
    }
]

In [5]:
# Configuramos MLflow para que guarde los resultados en la carpeta local
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment(PROJECT_NAME)

# BUCLE: Para cada experimento en la lista...
for i, exp in enumerate(experiments):
    run_name = exp["name"]       # Ej: "Exp1_Baseline"
    custom_params = exp["params"] # Ej: {"optimizer": "auto", ...}
    
    print(f"🧪 INICIANDO: {run_name}")
    
    # IMPORTANTE: Cargamos el modelo DE CERO en cada vuelta.
    # Si no hiciéramos esto, el Exp2 empezaría ya sabiendo lo que aprendió el Exp1.
    # Queremos que cada uno empiece "tonto" para comparar justamente.
    model = YOLO("yolov8n-seg.pt")
    
    # ENTRENAR
    results = model.train(
        data=str(parts_yaml_path),    # Dónde están los datos
        project="runs/segment",  # Carpeta raíz de salida
        name=run_name,           # Nombre de la subcarpeta
        epochs=EPOCHS,           # 20
        imgsz=IMG_SIZE,          # 640
        batch=BATCH_SIZE,        # 8
        device=0,                # Tu GPU GTX 1660 Ti
        exist_ok=True,           # Si ya existe la carpeta, sobrescríbela
        
        # EL TRUCO DE PYTHON: **custom_params
        # Esto "desempaqueta" el diccionario.
        # Es como escribir: optimizer="auto", lr0=0.01, mosaic=1.0
        **custom_params          
    )
    
    print(f"✅ {run_name} finalizado.")

2026/01/27 18:09:06 INFO mlflow.tracking.fluent: Experiment with name 'Lab_Vehicle_Parts' does not exist. Creating a new experiment.


🧪 INICIANDO: Exp1_Baseline
New https://pypi.org/project/ultralytics/8.4.8 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.7  Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\angel\Desktop\Proyecto\vehicle-damage-assesment\data\processed\parts_dataset\dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode

2026/01/27 18:10:16 INFO mlflow.tracking.fluent: Experiment with name 'runs/segment' does not exist. Creating a new experiment.


MLflow: logging run_id(7b50772f023245a7a822fe137b7f6f19) to runs\mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs\mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to C:\Users\angel\Desktop\Proyecto\vehicle-damage-assesment\notebooks\runs\segment\runs\segment\Exp1_Baseline
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss   sem_loss  Instances       Size
       1/20      3.34G      1.705      3.861      4.554      1.684          0        130        640: 100% ━━━━━━━━━━━━ 85/85 2.1it/s 40.0s0.4ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 3.2it/s 2.5s0.3s
                   all        120       1907     0.0235    0.00263     0.0128     0.0102     0.0235    0.00263     0.0128    0.00717

      Epoch    G

In [4]:
# Configuración rápida para prueba
TEST_EPOCHS = 3
BATCH_SIZE = 4  # Bajo para asegurar que no falte memoria
IMG_SIZE = 640

print("⏳ Cargando modelo YOLO...")
model = YOLO("yolov8n-seg.pt") 

# Iniciar MLflow
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("Notebook_Check")

print("🚀 Iniciando entrenamiento...")
results = model.train(
    data=str(parts_yaml_path),  # <--- Pasamos la ruta absoluta convertida a string
    epochs=TEST_EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    project="notebook_runs",
    name="test_parts_run",
    exist_ok=True
)

print("✅ ¡Entrenamiento finalizado con éxito!")

⏳ Cargando modelo YOLO...
🚀 Iniciando entrenamiento...
New https://pypi.org/project/ultralytics/8.4.8 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.7  Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\angel\Desktop\Proyecto\vehicle-damage-assesment\data\processed\parts_dataset\dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4

In [4]:
EPOCHS = 30       
IMG_SIZE = 832    
BATCH_SIZE = 2    
PROJECT_NAME = "Lab_Vehicle_Parts"

experiments = [
    
    {
        "name": "Exp4_SGD_Slow_IMG832",
        "params": {
            "optimizer": "SGD", # Stochastic Gradient Descent (Clásico, lento pero preciso)
            "lr0": 0.005,      # 0.005 es la mitad de 0.01. Aprende más despacio.
            "mosaic": 1.0,
        }
    }
]

In [5]:
# Configuramos MLflow para que guarde los resultados en la carpeta local
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment(PROJECT_NAME)

# BUCLE: Para cada experimento en la lista...
for i, exp in enumerate(experiments):
    run_name = exp["name"]       # Ej: "Exp1_Baseline"
    custom_params = exp["params"] # Ej: {"optimizer": "auto", ...}
    
    print(f"🧪 INICIANDO: {run_name}")
    
    # IMPORTANTE: Cargamos el modelo DE CERO en cada vuelta.
    # Si no hiciéramos esto, el Exp2 empezaría ya sabiendo lo que aprendió el Exp1.
    # Queremos que cada uno empiece "tonto" para comparar justamente.
    model = YOLO("yolov8n-seg.pt")
    
    # ENTRENAR
    results = model.train(
        data=str(parts_yaml_path),    # Dónde están los datos
        project="runs/segment",  # Carpeta raíz de salida
        name=run_name,           # Nombre de la subcarpeta
        epochs=EPOCHS,           # 20
        imgsz=IMG_SIZE,          # 640
        batch=BATCH_SIZE,        # 8
        device=0,                # Tu GPU GTX 1660 Ti
        exist_ok=True,           # Si ya existe la carpeta, sobrescríbela
        
        # EL TRUCO DE PYTHON: **custom_params
        # Esto "desempaqueta" el diccionario.
        # Es como escribir: optimizer="auto", lr0=0.01, mosaic=1.0
        **custom_params          
    )
    
    print(f"✅ {run_name} finalizado.")

🧪 INICIANDO: Exp4_SGD_Slow_IMG832
New https://pypi.org/project/ultralytics/8.4.8 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.7  Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\angel\Desktop\Proyecto\vehicle-damage-assesment\data\processed\parts_dataset\dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=832, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0

Traceback (most recent call last):
  File "c:\Users\angel\.conda\envs\vehicle-env\lib\site-packages\mlflow\store\tracking\file_store.py", line 302, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\Users\angel\.conda\envs\vehicle-env\lib\site-packages\mlflow\store\tracking\file_store.py", line 395, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\Users\angel\.conda\envs\vehicle-env\lib\site-packages\mlflow\store\tracking\file_store.py", line 1303, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\Users\angel\.conda\envs\vehicle-env\lib\site-packages\mlflow\store\tracking\file_store.py", line 1296, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\Users\angel\.conda\envs\vehicle-env\lib\site-packages\mlflow\utils\file_utils.py", line 313, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist.")
mlflow.e

MLflow: logging run_id(b3e1047f83494696bef56b0829607fbc) to runs\mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs\mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 832 train, 832 val
Using 8 dataloader workers
Logging results to C:\Users\angel\Desktop\Proyecto\vehicle-damage-assesment\notebooks\runs\segment\runs\segment\Exp4_SGD_Slow_IMG832
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss   sem_loss  Instances       Size
       1/30      2.83G      1.339      2.963       4.22      1.443          0         72        832: 100% ━━━━━━━━━━━━ 339/339 4.2it/s 1:21<0.2ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 30/30 5.2it/s 5.8s0.2s
                   all        120       1907      0.025      0.476     0.0795     0.0547     0.0239      0.445     0.0739      0.048

     